# MIMIC-IV Data Integration & Preprocessing Pipeline
## 30-Day Readmission Prediction

**Author**: Data Processing Pipeline  
**Date**: 2025-10-16  
**Objective**: Integrate multiple MIMIC-IV tables and generate features for 30-day readmission prediction

**Pipeline Steps**:
1. Load core tables (admissions, patients, diagnoses, procedures, etc.)
2. Define 30-day readmission target
3. Extract demographic, clinical, and temporal features
4. Handle missing values and data quality issues
5. Save cleaned dataset for modeling

## Step 1: Import Libraries and Setup

In [7]:
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Define data directories
mimic_data_dir = '/Users/yuchenzhou/Documents/duke/compsci526/final_proj/mimic_data'
output_dir = '/Users/yuchenzhou/Documents/duke/compsci526/final_proj/mimic_data/processed_data'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

print("✅ Libraries imported successfully")
print(f"📁 MIMIC data directory: {mimic_data_dir}")
print(f"📁 Output directory: {output_dir}")

✅ Libraries imported successfully
📁 MIMIC data directory: /Users/yuchenzhou/Documents/duke/compsci526/final_proj/mimic_data
📁 Output directory: /Users/yuchenzhou/Documents/duke/compsci526/final_proj/mimic_data/processed_data


## Step 2: Load Core Tables

In [8]:
print("=" * 80)
print("Loading MIMIC-IV Core Tables")
print("=" * 80)

# Load admissions
print("\n1️⃣ Loading admissions...")
admissions = pd.read_csv(os.path.join(mimic_data_dir, 'admissions.csv'))
print(f"   ✅ Admissions: {admissions.shape}")

# Load patients
print("\n2️⃣ Loading patients...")
patients = pd.read_csv(os.path.join(mimic_data_dir, 'patients.csv'))
print(f"   ✅ Patients: {patients.shape}")

# Load diagnoses_icd
print("\n3️⃣ Loading diagnoses_icd...")
diagnoses = pd.read_csv(os.path.join(mimic_data_dir, 'diagnoses_icd.csv'))
print(f"   ✅ Diagnoses: {diagnoses.shape}")

# Load procedures_icd
print("\n4️⃣ Loading procedures_icd...")
procedures = pd.read_csv(os.path.join(mimic_data_dir, 'procedures_icd.csv'))
print(f"   ✅ Procedures: {procedures.shape}")

# Load labevents (sample)
print("\n5️⃣ Loading labevents (sampling first 1M rows)...")
try:
    labevents = pd.read_csv(os.path.join(mimic_data_dir, 'labevents.csv'), nrows=1000000)
    print(f"   ✅ Labevents: {labevents.shape}")
except:
    print("   ⚠️ Labevents not found or too large, skipping...")
    labevents = None

print("\n✅ Core tables loaded successfully")

Loading MIMIC-IV Core Tables

1️⃣ Loading admissions...
   ✅ Admissions: (546028, 16)

2️⃣ Loading patients...
   ✅ Patients: (364627, 6)

3️⃣ Loading diagnoses_icd...
   ✅ Admissions: (546028, 16)

2️⃣ Loading patients...
   ✅ Patients: (364627, 6)

3️⃣ Loading diagnoses_icd...
   ✅ Diagnoses: (6364488, 5)

4️⃣ Loading procedures_icd...
   ✅ Diagnoses: (6364488, 5)

4️⃣ Loading procedures_icd...
   ✅ Procedures: (859655, 6)

5️⃣ Loading labevents (sampling first 1M rows)...
   ✅ Procedures: (859655, 6)

5️⃣ Loading labevents (sampling first 1M rows)...
   ✅ Labevents: (1000000, 16)

✅ Core tables loaded successfully
   ✅ Labevents: (1000000, 16)

✅ Core tables loaded successfully


## Step 3: Define 30-Day Readmission Target

In [9]:
print("=" * 80)
print("Defining 30-Day Readmission Target")
print("=" * 80)

# Convert datetime columns
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])

# Sort by subject_id and admission time
admissions_sorted = admissions.sort_values(['subject_id', 'admittime']).reset_index(drop=True)

# Calculate time to next admission
admissions_sorted['next_admittime'] = admissions_sorted.groupby('subject_id')['admittime'].shift(-1)
admissions_sorted['days_to_readmit'] = (
    admissions_sorted['next_admittime'] - admissions_sorted['dischtime']
).dt.total_seconds() / (24 * 3600)

# Define 30-day readmission
admissions_sorted['readmit_30d'] = (
    (admissions_sorted['days_to_readmit'] >= 0) & 
    (admissions_sorted['days_to_readmit'] <= 30)
).astype(int)

# Statistics
total_admissions = len(admissions_sorted)
readmitted = admissions_sorted['readmit_30d'].sum()
readmit_rate = (readmitted / total_admissions) * 100

print(f"\n📊 Readmission Statistics:")
print(f"   Total admissions: {total_admissions:,}")
print(f"   30-day readmissions: {readmitted:,}")
print(f"   Readmission rate: {readmit_rate:.2f}%")

print("\n✅ Target variable created")

Defining 30-Day Readmission Target

📊 Readmission Statistics:
   Total admissions: 546,028
   30-day readmissions: 109,345
   Readmission rate: 20.03%

✅ Target variable created

📊 Readmission Statistics:
   Total admissions: 546,028
   30-day readmissions: 109,345
   Readmission rate: 20.03%

✅ Target variable created


## Step 4: Extract Demographic Features

In [10]:
print("=" * 80)
print("Extracting Demographic Features")
print("=" * 80)

# Merge with patients table
df = admissions_sorted.merge(patients, on='subject_id', how='left')

# Calculate age at admission
df['anchor_year_group'] = df['anchor_year_group'].str.split('-').str[0].astype(int)
df['age'] = df['anchor_year_group'] + df['anchor_age'] - df['anchor_year']

# Gender encoding
df['gender_M'] = (df['gender'] == 'M').astype(int)
df['gender_F'] = (df['gender'] == 'F').astype(int)

# Admission type encoding
admission_types = pd.get_dummies(df['admission_type'], prefix='admission')
df = pd.concat([df, admission_types], axis=1)

# Admission location encoding
admission_loc = pd.get_dummies(df['admission_location'], prefix='admit_loc')
df = pd.concat([df, admission_loc], axis=1)

# Insurance encoding
insurance = pd.get_dummies(df['insurance'], prefix='insurance')
df = pd.concat([df, insurance], axis=1)

# Marital status encoding  
marital = pd.get_dummies(df['marital_status'], prefix='marital')
df = pd.concat([df, marital], axis=1)

# Race/Ethnicity encoding
race = pd.get_dummies(df['race'], prefix='race')
df = pd.concat([df, race], axis=1)

print(f"\n✅ Demographic features extracted")
print(f"   Current shape: {df.shape}")

Extracting Demographic Features

✅ Demographic features extracted
   Current shape: (546028, 89)

✅ Demographic features extracted
   Current shape: (546028, 89)


## Step 5: Extract Clinical Features

In [11]:
print("=" * 80)
print("Extracting Clinical Features")
print("=" * 80)

# Length of stay
df['los_days'] = (df['dischtime'] - df['admittime']).dt.total_seconds() / (24 * 3600)

# Hospital expire flag
df['hospital_expire_flag'] = df['hospital_expire_flag'].fillna(0).astype(int)

# Number of diagnoses
print("\n1️⃣ Counting diagnoses per admission...")
diag_counts = diagnoses.groupby('hadm_id').size().reset_index(name='num_diagnoses')
df = df.merge(diag_counts, on='hadm_id', how='left')
df['num_diagnoses'] = df['num_diagnoses'].fillna(0)

# Number of procedures
print("\n2️⃣ Counting procedures per admission...")
proc_counts = procedures.groupby('hadm_id').size().reset_index(name='num_procedures')
df = df.merge(proc_counts, on='hadm_id', how='left')
df['num_procedures'] = df['num_procedures'].fillna(0)

# Primary diagnosis (first diagnosis code)
print("\n3️⃣ Extracting primary diagnosis...")
primary_diag = diagnoses.sort_values(['hadm_id', 'seq_num']).groupby('hadm_id').first().reset_index()
primary_diag = primary_diag[['hadm_id', 'icd_code']]
primary_diag.columns = ['hadm_id', 'primary_icd_code']
df = df.merge(primary_diag, on='hadm_id', how='left')

# Discharge location encoding
discharge_loc = pd.get_dummies(df['discharge_location'], prefix='discharge')
df = pd.concat([df, discharge_loc], axis=1)

print(f"\n✅ Clinical features extracted")
print(f"   Current shape: {df.shape}")

Extracting Clinical Features

1️⃣ Counting diagnoses per admission...

2️⃣ Counting procedures per admission...

3️⃣ Extracting primary diagnosis...

2️⃣ Counting procedures per admission...

3️⃣ Extracting primary diagnosis...

✅ Clinical features extracted
   Current shape: (546028, 106)

✅ Clinical features extracted
   Current shape: (546028, 106)


## Step 6: Extract Temporal Features

In [12]:
print("=" * 80)
print("Extracting Temporal Features")
print("=" * 80)

# Admission year, month, day of week
df['admit_year'] = df['admittime'].dt.year
df['admit_month'] = df['admittime'].dt.month
df['admit_dow'] = df['admittime'].dt.dayofweek  # 0=Monday, 6=Sunday
df['admit_hour'] = df['admittime'].dt.hour

# Weekend admission
df['admit_weekend'] = (df['admit_dow'] >= 5).astype(int)

# Season
df['admit_season'] = pd.cut(df['admit_month'], 
                              bins=[0, 3, 6, 9, 12],
                              labels=['Winter', 'Spring', 'Summer', 'Fall'])
season_dummies = pd.get_dummies(df['admit_season'], prefix='season')
df = pd.concat([df, season_dummies], axis=1)

# Previous admissions count
print("\n1️⃣ Counting previous admissions...")
df_sorted = df.sort_values(['subject_id', 'admittime'])
df_sorted['prev_admissions'] = df_sorted.groupby('subject_id').cumcount()
df['prev_admissions'] = df_sorted['prev_admissions']

# Days since last admission
print("\n2️⃣ Calculating days since last admission...")
df_sorted['prev_dischtime'] = df_sorted.groupby('subject_id')['dischtime'].shift(1)
df_sorted['days_since_last_admission'] = (
    df_sorted['admittime'] - df_sorted['prev_dischtime']
).dt.total_seconds() / (24 * 3600)
df['days_since_last_admission'] = df_sorted['days_since_last_admission'].fillna(-1)

print(f"\n✅ Temporal features extracted")
print(f"   Current shape: {df.shape}")

Extracting Temporal Features

1️⃣ Counting previous admissions...

2️⃣ Calculating days since last admission...

✅ Temporal features extracted
   Current shape: (546028, 118)

2️⃣ Calculating days since last admission...

✅ Temporal features extracted
   Current shape: (546028, 118)


## Step 7: Data Cleaning and Quality Checks

In [13]:
print("=" * 80)
print("Data Cleaning and Quality Checks")
print("=" * 80)

print(f"\n📊 Initial dataset: {df.shape}")

# Remove rows with missing target
print("\n1️⃣ Removing rows with missing target...")
initial_rows = len(df)
df = df[df['readmit_30d'].notna()]
removed = initial_rows - len(df)
print(f"   Removed {removed} rows with missing target")

# Remove unrealistic ages
print("\n2️⃣ Filtering age range...")
df = df[(df['age'] >= 18) & (df['age'] <= 120)]
print(f"   Kept admissions for ages 18-120")

# Remove negative or zero LOS
print("\n3️⃣ Filtering length of stay...")
df = df[df['los_days'] > 0]
print(f"   Removed admissions with LOS <= 0")

# Handle missing values
print("\n4️⃣ Handling missing values...")
print(f"   Missing values per column:")
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    for col, count in missing.head(10).items():
        pct = (count / len(df)) * 100
        print(f"      {col}: {count} ({pct:.2f}%)")
else:
    print("      No missing values!")

# Fill numeric missing with median
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].median(), inplace=True)

# Fill categorical missing with 'Unknown'
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna('Unknown', inplace=True)

print(f"\n✅ Data cleaning complete")
print(f"   Final dataset: {df.shape}")

Data Cleaning and Quality Checks

📊 Initial dataset: (546028, 118)

1️⃣ Removing rows with missing target...
   Removed 0 rows with missing target

2️⃣ Filtering age range...
   Kept admissions for ages 18-120

3️⃣ Filtering length of stay...
   Removed admissions with LOS <= 0

4️⃣ Handling missing values...
   Missing values per column:
      No missing values!

✅ Data cleaning complete
   Final dataset: (0, 118)


## Step 8: Select Final Features and Save

In [14]:
print("=" * 80)
print("Selecting Features and Saving Dataset")
print("=" * 80)

# Select key columns to keep
columns_to_keep = ['subject_id', 'hadm_id', 'readmit_30d', 'age', 'los_days', 
                   'num_diagnoses', 'num_procedures', 'prev_admissions',
                   'days_since_last_admission', 'admit_year', 'admit_month', 
                   'admit_dow', 'admit_hour', 'admit_weekend', 'hospital_expire_flag',
                   'primary_icd_code']

# Add all one-hot encoded columns
encoded_prefixes = ['gender', 'admission', 'admit_loc', 'insurance', 'marital', 
                    'race', 'discharge', 'season']
for prefix in encoded_prefixes:
    encoded_cols = [col for col in df.columns if col.startswith(prefix + '_')]
    columns_to_keep.extend(encoded_cols)

# Keep only columns that exist
columns_to_keep = [col for col in columns_to_keep if col in df.columns]

# Create final dataset
df_final = df[columns_to_keep].copy()

print(f"\n📊 Final Dataset Summary:")
print(f"   Shape: {df_final.shape}")
print(f"   Features: {df_final.shape[1] - 1}")  # Excluding target
print(f"   Target variable: readmit_30d")
print(f"   Readmission rate: {df_final['readmit_30d'].mean()*100:.2f}%")

# Save to CSV
output_path = os.path.join(output_dir, 'readmission_features_cleaned.csv')
df_final.to_csv(output_path, index=False)
print(f"\n✅ Dataset saved to: {output_path}")

# Display first few rows
print(f"\n📋 First 5 rows:")
print(df_final.head())

Selecting Features and Saving Dataset

📊 Final Dataset Summary:
   Shape: (0, 101)
   Features: 100
   Target variable: readmit_30d
   Readmission rate: nan%

✅ Dataset saved to: /Users/yuchenzhou/Documents/duke/compsci526/final_proj/mimic_data/processed_data/readmission_features_cleaned.csv

📋 First 5 rows:
Empty DataFrame
Columns: [subject_id, hadm_id, readmit_30d, age, los_days, num_diagnoses, num_procedures, prev_admissions, days_since_last_admission, admit_year, admit_month, admit_dow, admit_hour, admit_weekend, hospital_expire_flag, primary_icd_code, gender_M, gender_F, admission_type, admission_location, admission_AMBULATORY OBSERVATION, admission_DIRECT EMER., admission_DIRECT OBSERVATION, admission_ELECTIVE, admission_EU OBSERVATION, admission_EW EMER., admission_OBSERVATION ADMIT, admission_SURGICAL SAME DAY ADMISSION, admission_URGENT, admit_loc_AMBULATORY SURGERY TRANSFER, admit_loc_CLINIC REFERRAL, admit_loc_EMERGENCY ROOM, admit_loc_INFORMATION NOT AVAILABLE, admit_loc_IN

## Step 9: Generate Summary Report

In [15]:
print("=" * 80)
print("Generating Summary Report")
print("=" * 80)

report = f"""
{'='*100}
MIMIC-IV Data Integration & Preprocessing Report
{'='*100}

Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

【Dataset Overview】
{'-'*100}
Total admissions: {len(df_final):,}
Total features: {df_final.shape[1] - 1}
Target variable: readmit_30d (30-day readmission)

【Target Distribution】
{'-'*100}
No readmission (0): {(df_final['readmit_30d']==0).sum():,} ({(df_final['readmit_30d']==0).mean()*100:.2f}%)
Readmitted (1): {(df_final['readmit_30d']==1).sum():,} ({(df_final['readmit_30d']==1).mean()*100:.2f}%)

【Feature Categories】
{'-'*100}
Demographic features:
  - Age, gender, race, marital status, insurance
  
Clinical features:
  - Length of stay, number of diagnoses, number of procedures
  - Primary diagnosis code, hospital expire flag
  - Admission type, admission location, discharge location
  
Temporal features:
  - Previous admissions count
  - Days since last admission
  - Admission time features (year, month, day of week, hour)
  - Weekend admission, season

【Data Quality】
{'-'*100}
Missing values: {df_final.isnull().sum().sum()}
Duplicate rows: {df_final.duplicated().sum()}

【Age Distribution】
{'-'*100}
Mean age: {df_final['age'].mean():.1f} years
Median age: {df_final['age'].median():.1f} years
Age range: {df_final['age'].min():.0f} - {df_final['age'].max():.0f} years

【Length of Stay】
{'-'*100}
Mean LOS: {df_final['los_days'].mean():.1f} days
Median LOS: {df_final['los_days'].median():.1f} days

【Output Files】
{'-'*100}
Cleaned dataset: {output_path}

{'='*100}
"""

print(report)

# Save report
report_path = os.path.join(output_dir, 'preprocessing_report.txt')
with open(report_path, 'w') as f:
    f.write(report)

print(f"✅ Report saved to: {report_path}")

Generating Summary Report

MIMIC-IV Data Integration & Preprocessing Report

Generated: 2025-10-18 23:33:27

【Dataset Overview】
----------------------------------------------------------------------------------------------------
Total admissions: 0
Total features: 100
Target variable: readmit_30d (30-day readmission)

【Target Distribution】
----------------------------------------------------------------------------------------------------
No readmission (0): 0 (nan%)
Readmitted (1): 0 (nan%)

【Feature Categories】
----------------------------------------------------------------------------------------------------
Demographic features:
  - Age, gender, race, marital status, insurance

Clinical features:
  - Length of stay, number of diagnoses, number of procedures
  - Primary diagnosis code, hospital expire flag
  - Admission type, admission location, discharge location

Temporal features:
  - Previous admissions count
  - Days since last admission
  - Admission time features (year, mont

## 🎉 Pipeline Complete!

**Next Steps:**
1. Open `baseline_models.ipynb` to build and evaluate baseline models
2. The cleaned dataset is ready at: `/Users/yuchenzhou/Documents/duke/compsci526/final_proj/mimic_data/processed_data/readmission_features_cleaned.csv`
3. Review the preprocessing report for data quality insights

**Key Outputs:**
- ✅ `readmission_features_cleaned.csv` - Cleaned dataset for modeling
- ✅ `preprocessing_report.txt` - Data quality and statistics report